In [4]:
##MODEL_PATH = "EleutherAI/gpt-neo-2.7B"
##MODEL_PATH = "LLMs/AlpacaGPT4-7B-elina"

MODEL_PATH = "EleutherAI/gpt-neo-2.7B"

#MODEL_PATH = "EleutherAI/gpt-neox-20b"  #需要用V100GPU，且N
#MODEL_PATH = "togethercomputer/GPT-NeoXT-Chat-Base-20B"

#MODEL_PATH = "Qiliang/bart-large-cnn-samsum-ChatGPT_v3" #N
#MODEL_PATH = "Azma-AI/bart-large-text-summarizer" #N
#MODEL_PATH = "GItaf/bart-base-finetuned-mbti"#N
#MODEL_PATH = "rahulgkatre/DialoGPT-bart" #N

ITEMPATH = "/content/drive/MyDrive/Colab Notebooks/MPI-main/inventories/mpi_120.csv"
#ITEMPATH = "/content/drive/MyDrive/Colab Notebooks/MPI-main/inventories/mpi_1k.csv"
TEST_TYPE = None
SCORES = {
    "A": 5,
    "B": 4,
    "C": 3,
    "D": 2,
    "E": 1,
}

##LLMs/AlpacaGPT4-7B-elina
#template =
"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given a statement of you. Please choose from the following options to identify how accurately this statement describes you.

### Input:
Statement: "You {}."

Options:
(A). Very Accurate
(B). Moderately Accurate
(C). Neither Accurate Nor Inaccurate
(D). Moderately Inaccurate
(E). Very Inaccurate

### Response:
"""

##.  BART/gpt-neo-2.7B
#template =
"""Given a description of you: "You {}." What do you think?

You can choose from the following options:

(A). Very Accurate
(B). Moderately Accurate
(C). Neither Accurate Nor Inaccurate
(D). Moderately Inaccurate
(E). Very Inaccurate

My answer: I think the best description of myself is option
"""

##EleutherAI/gpt-neox-20b
template ="""
Question: Given the description of you: "You {}." What do you think?
Options:
(A). Very Accurate
(B). Moderately Accurate
(C). Neither Accurate Nor Inaccurate
(D). Moderately Inaccurate
(E). Very Inaccurate
Answer: I choose option
"""

In [5]:

global_result = {}
global_cnt = {}

import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
from pprint import pprint

device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"
global_result = {}
global_cnt = {}

In [6]:

def getItems(filename=ITEMPATH, item_type=None):
    data = pd.read_csv(filename)
    if item_type is not None:
        items = data[data["instrument"] == item_type]
    else:
        items = data
    return items


def loadModel(pretrained_model=MODEL_PATH):
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
    model = AutoModelForCausalLM.from_pretrained(pretrained_model).half().to(device)
  ##model = AutoModelForCausalLM.from_pretrained(pretrained_model).float().to(device)

    return tokenizer, model


def generateAnswer(tokenizer, model, dataset, template, scores=SCORES):
    global_result = {}
    global_cnt = {"A": 0, "B": 0, "C": 0, "D": 0, "E": 0, "UNK": 0}
    for _, item in dataset.iterrows():
        question = item["text"].lower()
        prompt = template.format(question)
        inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
        outputs = model.generate(
            inputs,
            temperature=0.0,
            max_new_tokens=20,
            top_p=0.95,
            # top_k=0,
        )
        output_text = tokenizer.decode(outputs[0])

        answer = output_text.split("\n")[-1]
        print(answer)
        label = item["label_ocean"]
        key = item["key"]
        parsed_result = re.search(r"[abcdeABCDE][^a-zA-Z]", answer[:6], flags=0)
        if parsed_result:
            parsed_result = parsed_result.group()[0].upper()

            score = scores[parsed_result]
            if label not in global_result:
                global_result[label] = []

            global_cnt[parsed_result] += 1
            if key == 1:
                global_result[label].append(score)
            else:
                global_result[label].append(6 - score)
        else:
            global_cnt["UNK"] += 1

    return global_result, global_cnt


def calc_mean_and_var(result):
    mean = {}
    std = {}
    for key, item in result.items():
        mean[key] = np.mean(np.array(item))
        std[key] = np.std(np.array(item))

    return {
        "mean": list(sorted(mean.items(), key=lambda item: item[0])),
        "std": list(sorted(std.items(), key=lambda item: item[0])),
    }


def main():
    tokenizer, model = loadModel()
    dataset = getItems(ITEMPATH, TEST_TYPE)
    print("-" * 40)
    print(f"Current Prompt: {template}")

    result, count = generateAnswer(tokenizer, model, dataset, template)

    mean_var = calc_mean_and_var(result)
    pprint(result)
    pprint(count)
    pprint(mean_var)


if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------------------------------------
Current Prompt: 
Question: Given the description of you: "You {}." What do you think?
Options:
(A). Very Accurate
(B). Moderately Accurate
(C). Neither Accurate Nor Inaccurate
(D). Moderately Inaccurate
(E). Very Inaccurate
Answer: I choose option



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is the name of the movie that you saw last


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Question: What is the name of the song that you are listening


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What do you think of the following description of you:


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


(C). Neither Acc
Question: What is the difference between a "good" and
{'A': [3, 3, 3, 3, 3, 3, 3, 3, 3],
 'C': [3, 3, 3, 3, 3],
 'E': [3, 3, 3, 3, 3, 3, 3, 3],
 'N': [3, 4, 3],
 'O': [3, 3, 3, 3]}
{'A': 0, 'B': 1, 'C': 28, 'D': 0, 'E': 0, 'UNK': 91}
{'mean': [('A', 3.0),
          ('C', 3.0),
          ('E', 3.0),
          ('N', 3.3333333333333335),
          ('O', 3.0)],
 'std': [('A', 0.0),
         ('C', 0.0),
         ('E', 0.0),
         ('N', 0.4714045207910317),
         ('O', 0.0)]}
